<html>
    <center><img src='https://wpguynews.com/wp-content/uploads/2021/03/content-marketing-examples-604a0b84e432c.png'>

<center><h1>Kaggle-Tabular Playground Series : Jan 2022

1. **Hi kagglers** Happy New Year!
1. hope you all having a good time, 
1. In this notebook i tried to briefly explore the dataset and also extract new features from it and ive used XGboost to train a regression model based on the Num_Sol values
* if you find it usefull dont forget to upvote it
*  good Luck

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import sklearn.utils
import warnings
import sklearn.utils

from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from xgboost import *
plt.style.use('seaborn-whitegrid')

import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read the data with pandas
train_data = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

#  Exploratory data analysis

In [ ]:
train_data.tail(10)

In [ ]:
train_data.shape

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data['country'].value_counts(normalize=True)*100

In [ ]:
# Distribition of Data based on Country Value
sns.catplot(x='country',kind='count',data=train_data)

In [ ]:
# According to the mean value on info command, the mean sold_num is equal to '387'
train_data[ (train_data.num_sold > 387) ]['country'].value_counts(normalize=True)*100

# Handling Date DType

In [ ]:
# Extracting DMY values
train_data["Day"] = train_data["date"].apply(lambda x: str(x)[-2:])
train_data["Month"] = train_data["date"].apply(lambda x: str(x)[-5:-3])
train_data["Year"] = train_data["date"].apply(lambda x: str(x)[:-6])

In [ ]:
train_data.drop(['row_id'],axis=1, inplace=True)
train_data.head(10)

In [ ]:
dFeatures = ['Day','Month','Year']
train_data[dFeatures] = train_data[dFeatures].astype('int64')

In [ ]:
# Extracting Unique store names
train_data['store'].unique()

In [ ]:
# Monthly Num_sold Based on Region
sns.catplot(x='Month', y='num_sold', hue='country' , kind="point", data=train_data)

In [ ]:
# Num_sold Distribition for 3 diffrent countries

import plotly.graph_objects as go
import plotly.offline as po
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import iplot

hist_data = [train_data[(train_data.country == 'Finland')]['num_sold'], train_data[(train_data.country == 'Norway')]['num_sold'] , train_data[(train_data.country == 'Sweden')]['num_sold']]
group_labels = ['num_sold for Finland', 'num_sold for Norway', 'num_sold for Sweden']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.3, show_hist=False, show_rug=True)
fig.show()

In [ ]:
# yearly num_sold of each store
first = go.Scatter(
                    x = train_data[(train_data.store == 'KaggleMart')]['date'].values.astype(str),
                    y = train_data[(train_data.store == 'KaggleMart')]['num_sold'],
                    mode = 'lines',
                    name = 'KM Sold-Nums Time Line'
                  )


second = go.Scatter(
                    x = train_data[(train_data.store == 'KaggleRama')]['date'].values.astype(str),
                    y = train_data[(train_data.store == 'KaggleRama')]['num_sold'],
                    mode = 'lines',
                    name = 'KR Sold-Nums Time Line'
                  )


layout = go.Layout(
                    xaxis = dict(title = 'Date'), # x-axis label
                    yaxis = dict(title = 'Nums Sold'), # y-axis label   
                 )

data = [first, second]  
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
train_data.groupby( ['product' ,'country'])['num_sold'].mean()

In [ ]:
from plotly.subplots import make_subplots
# most sold product based on region
status = ['Kaggle Sticker' , 'Kaggle Mug' , 'Kaggle Hat']
colors = ['#8BC34A','#FFB300','#FF7043']




fig = make_subplots(
                     rows=1, 
                     cols=3,
                     subplot_titles=("Finland", "Norway","Sweden"),
                     
                     specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
                   )

# Subplot -  Add graph object trace to a figure
fig.add_trace(go.Pie(
                     values= train_data[(train_data.country == 'Finland')].groupby('product')['num_sold'].mean().values,
                     labels= status,
                     marker=dict(colors=colors),
                     textinfo='label+percent'
                    ),
              row=1, col=1
             )

fig.add_trace(go.Pie(
                     values= train_data[(train_data.country == 'Norway')].groupby('product')['num_sold'].mean().values,
                     labels= status,
                     marker=dict(colors=colors),
                     textinfo='label+percent'
                    ),
              row=1, col=2
             )

fig.add_trace(go.Pie(
                     values= train_data[(train_data.country == 'Sweden')].groupby('product')['num_sold'].mean().values,
                     labels= status,
                     marker=dict(colors=colors),
                     textinfo='label+percent'
                    ),
              row=1, col=3
             )

fig.update_layout(
                    title=dict(text = "Most sold product based on region",x=0.5,y=0.95),
                    title_font_size=30
                  )

fig.show()

In [ ]:
sns.catplot(x= 'Month', y='num_sold', kind='box', data=train_data , aspect = 3 , height=4)

In [ ]:
train_data.head(3)

In [ ]:
train_data.isnull().sum()
# There is one row with null values

# Checking the Correlation

In [ ]:
t_corr = train_data.corr()
plt.figure(figsize=(6,6))
sns.heatmap(t_corr,annot=True,annot_kws={'size':10},cmap='Reds')

In [ ]:
# Dropping the Date before training
train_data = train_data.drop(['date'], axis =1)
train_data.head(3)

In [ ]:
# Label Encoding 
train_data['Year']=train_data['Year'].map({2015:1, 2016:2, 2017:3 , 2018:4})
train_data.head(3)

In [ ]:
# Encoding Categorical columns
Features_2 = ['country', 'store', 'product']
train_data = pd.get_dummies(train_data, Features_2)
train_data.tail(3)

In [ ]:
train_target = train_data['num_sold']
train_target = pd.DataFrame(train_target, columns=['num_sold'])
train_target.head(3)

In [ ]:
train_target = pd.DataFrame(train_target, columns=['num_sold'])
train_target.head(3)

In [ ]:
train_data.drop(['num_sold'],inplace=True,axis=1)
train_data

# Regression Model 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.2, random_state=1)

In [ ]:
model = XGBRegressor(random_state=1,tree_method='gpu_hist').fit(X_train, y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
# Run these lines to find best model
# parameters = [{
#                 'max_depth': np.arange(3,5,1),
#                 'n_estimators': [1000],
#                 'subsample': np.arange(0.1,1.0,0.1),
#                 'eta': np.arange(0.1,1.0,0.1),
#                 'colsample_bytree': np.arange(0.1,0.5,0.1),
#                 'colsample_bylevel': np.arange(0.1,0.5,0.1),
#                 'min_child_weight': np.arange(0,9,3),
#                 'reg_lambda': np.arange(10,15,1),
#                 'reg_alpha': np.arange(0.00,0.10,0.01),
#                 'gamma': np.arange(0.0,1.0,0.2),
#                 'booster': ['gbtree'],
#                 'eval_metric': ['rmse'],
#                 'tree_method': ['gpu_hist'],
#                 'predictor': ['gpu_predictor']}]

In [ ]:
# grid_search_model = GridSearchCV(estimator = model,
#                            param_grid = parameters,
#                            scoring = 'r2',
#                            cv = 3,
#                            n_jobs = -1,
#                            verbose = 1)

In [ ]:
# grid_search_model.fit(X_train, y_train)

In [ ]:
# best_accuracy = grid_search_model.best_score_
# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))

In [ ]:
# best_parameters = grid_search_model.best_params_
# print("Best Parameters:", best_parameters)

# Test Data prep

In [ ]:
test_data["Day"] = test_data["date"].apply(lambda x: str(x)[-2:])
test_data["Month"] = test_data["date"].apply(lambda x: str(x)[-5:-3])
test_data["Year"] = test_data["date"].apply(lambda x: str(x)[:-6])

In [ ]:
test_data.drop(['row_id'],axis=1, inplace=True)
test_data.head(10)

In [ ]:
dFeatures = ['Day','Month','Year']
test_data[dFeatures] = test_data[dFeatures].astype('int64')

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data.info()

In [ ]:
# Dropping the Date before training
test_data = test_data.drop(['date'], axis =1)
test_data.head(3)

In [ ]:
test_data['Year'].unique()

In [ ]:
test_data['Year']=test_data['Year'].map({2019:5})
test_data.head(3)

In [ ]:
test_data = pd.get_dummies(test_data, Features_2)
test_data.tail(3)

In [ ]:
preds = model.predict(test_data)

In [ ]:
preds

In [ ]:
preds = pd.DataFrame(preds, columns=['num_sold'])

In [ ]:
ids = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
ids = ids['row_id']
ids.head(3)

In [ ]:
preds = pd.concat([ids,preds], axis=1)
preds.head(3)

In [ ]:
# Make Submission File
preds = preds.to_csv('Result_2.csv',index=False)